In [11]:
!guardrails hub install hub://guardrails/valid_length --quiet
!guardrails hub install hub://guardrails/two_words --quiet
!guardrails hub install hub://guardrails/valid_range --quiet
!guardrails hub install hub://guardrails/lowercase --quiet
!guardrails hub install hub://guardrails/one_line --quiet

%pip install pypdfium2

Installing hub://guardrails/valid_length...
✅Successfully installed guardrails/valid_length!


Installing hub://guardrails/two_words...
✅Successfully installed guardrails/two_words!


Installing hub://guardrails/valid_range...
✅Successfully installed guardrails/valid_range!


Installing hub://guardrails/lowercase...
✅Successfully installed guardrails/lowercase!


Installing hub://guardrails/one_line...
✅Successfully installed guardrails/one_line!


Note: you may need to restart the kernel to use updated packages.


# Extracting entities from a Terms of Service document

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/src/examples/extracting_entities.ipynb).

In this example, we will use Guardrails to extract key information from a Terms-of-Service document.

## Objective

We want to extract structured information about all fees and interest rates associated with the Chase credit card.

## Step 0: Download PDF and load it as string

To get started, download the document from [here](https://github.com/ShreyaR/guardrails/blob/main/docs/src/examples/data/chase_card_agreement.pdf) and save it in `data/chase_card_agreement.pdf`.

Guardrails has some built-in functions to help with common tasks. Here, we will use the `read_pdf` function to load the PDF as a string.

In [2]:
import guardrails as gd

from rich import print

content = gd.docs_utils.read_pdf("data/chase_card_agreement.pdf")

print(f"Chase Credit Card Document:\n\n{content[:275]}\n...")

/Users/dtam/.pyenv/versions/3.12.3/envs/litellm/lib/python3.12/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


Chase Credit Card Document:

2/25/23, 7:59 PM about:blank
about:blank 1/4
PRICING INFORMATION
INTEREST RATES AND INTEREST CHARGES
Purchase Annual
Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.
After that, 19.49%. This APR will vary with the market based on the Prim
...

## Step 1: Create the Pydantic RAIL model
Here, we request:

1. A list of the fees associated with the card. We ask for sub-information, each with its own quality criteria and corrective action.
2. A object (i.e. key-value pairs) for the interest.

In [3]:
from guardrails.hub import LowerCase, TwoWords, OneLine
from pydantic import BaseModel, Field

prompt = """
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

${document}

${gr.complete_xml_suffix_v2}
"""


class Fee(BaseModel):
    name: str = Field(validators=[LowerCase(on_fail="fix"), TwoWords(on_fail="reask")])
    explanation: str = Field(validators=[OneLine(on_fail="noop")])
    value: float = Field(description="The fee amount in USD or as a percentage.")


class AccountFee(BaseModel):
    account_type: str = Field(validators=[LowerCase(on_fail="fix")])
    rate: float = Field(
        description="The annual percentage rate (APR) for the account type."
    )


class CreditCardAgreement(BaseModel):
    fees: list[Fee] = Field(
        description="What fees and charges are associated with my account?"
    )
    interest_rates: list[AccountFee] = Field(
        description="What are the interest rates offered by the bank on different kinds of accounts and products?"
    )

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the Pydantic RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [4]:
guard = gd.Guard.for_pydantic(output_class=CreditCardAgreement)

As we can see, a few formatters weren't supported. These formatters won't be enforced in the output, but this information can still be used to generate a prompt.

We see the prompt that will be sent to the LLM. The `{document}` is substituted with the user provided value at runtime.

## Step 3: Wrap the LLM API call with `Guard`

In [8]:
# Add your OPENAI_API_KEY as an environment variable if it's not already set
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

raw_llm_response, validated_response, *rest = guard(
    messages=[{"role": "user", "content": prompt}],
    prompt_params={"document": content[:6000]},
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0,
)

/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [9]:
print(validated_response)

{
    'fees': [
        {
            'name': 'annual membership',
            'explanation': 'No annual membership fee is charged for this account.',
            'value': 0.0
        },
        {
            'name': 'my chase',
            'explanation': 'Monthly fee of 0% of the amount of each eligible purchase transaction during the 0% 
Intro Purchase APR period, then 1.72% after.',
            'value': 1.72
        },
        {
            'name': 'balance transfer',
            'explanation': 'Intro fee of either $5 or 3% of the amount of each transfer, whichever is greater, on 
transfers made within 60 days of account opening. After that: Either $5 or 5% of the amount of each transfer, 
whichever is greater.',
            'value': 5.0
        },
        {
            'name': 'cash advance',
            'explanation': 'Either $10 or 5% of the amount of each transaction, whichever is greater.',
            'value': 10.0
        },
        {
            'name': 'foreign transaction',
            'explanation': '3% of the amount of each transaction in U.S. dollars.',
            'value': 3.0
        },
        {'name': 'late payment', 'explanation': 'Up to $40 for late payments.', 'value': 40.0},
        {'name': 'return payment', 'explanation': 'Up to $40 for returned payments.', 'value': 40.0}
    ],
    'interest_rates': [
        {'account_type': 'purchase', 'rate': 0.0},
        {'account_type': 'balance transfer', 'rate': 0.0},
        {'account_type': 'cash advance', 'rate': 29.49},
        {'account_type': 'penalty', 'rate': 29.99},
        {'account_type': 'my chase loan', 'rate': 19.49}
    ]
}

In [10]:
guard.history.last.tree

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
│   │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │ │
│   │ │ ┃ Role ┃ Content                                                                                      ┃ │ │
│   │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │ │
│   │ │ │ user │                                                                                              │ │ │
│   │ │ │      │ Given the following document, answer the following questions. If the answer doesn't exist in │ │ │
│   │ │ │      │ the document, enter 'None'.                                                                  │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ 2/25/23, 7:59 PM about:blank                                                                 │ │ │
│   │ │ │      │ about:blank 1/4                                                                              │ │ │
│   │ │ │      │ PRICING INFORMATION                                                                          │ │ │
│   │ │ │      │ INTEREST RATES AND INTEREST CHARGES                                                          │ │ │
│   │ │ │      │ Purchase Annual                                                                              │ │ │
│   │ │ │      │ Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.        │ │ │
│   │ │ │      │ After that, 19.49%. This APR will vary with the market based on the Prime                    │ │ │
│   │ │ │      │ Rate.                                                                                        │ │ │
│   │ │ │      │ a                                                                                            │ │ │
│   │ │ │      │ My Chase Loan                                                                                │ │ │
│   │ │ │      │ SM APR 19.49%. This APR will vary with the market based on the Prime Rate.                   │ │ │
│   │ │ │      │ a                                                                                            │ │ │
│   │ │ │      │ Promotional offers with fixed APRs and varying durations may be available from               │ │ │
│   │ │ │      │ time to time on some accounts.                                                               │ │ │
│   │ │ │      │ Balance Transfer APR 0% Intro APR for the first 18 months that your Account is open.         │ │ │
│   │ │ │      │ After that, 19.49%. This APR will vary with the market based on the Prime                    │ │ │
│   │ │ │      │ Rate.                                                                                        │ │ │
│   │ │ │      │ a                                                                                            │ │ │
│   │ │ │      │ Cash Advance APR 29.49%. This APR will vary with the market based on the Prime Rate.         │ │ │
│   │ │ │      │ b                                                                                            │ │ │
│   │ │ │      │ Penalty APR and When                                                                         │ │ │
│   │ │ │      │ It Applies                                                                                   │ │ │
│   │ │ │      │ Up to 29.99%. This APR will vary with the market based on the Prime Rate.                    │ │ │
│   │ │ │      │ c                                                                                            │ │ │
│   │ │ │      │ We may apply the Penalty APR to your account if you:                                         │ │ │
│   │ │ │      │ fail to make a Minimum Payment by 